### Checking Versions

First, let's check which versions of Python and Spark we're using:

In [1]:
# Print Python and Spark versions
import sys
from pyspark.sql import SparkSession

# Create a SparkSession first
spark = SparkSession.builder.appName("Showdown-Replay-Exploration").getOrCreate()

print(f"Python version: {sys.version}")
print(f"Spark version: {spark.version}")

Python version: 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]
Spark version: 3.5.0


### Step 1: Setup and Load JSON Files

We will load only the Regulation G Json files.

In [2]:
# Use spark to read files then partition after to speed up processing

# Read all JSON files from the directory recursively
logs_df = spark.read \
    .option("recursiveFileLookup", "true") \
    .option("pathGlobFilter", "*.json") \
    .json("../../data/replays/gen9vgc2025regg/")  # adjust path accordingly

# Optionally repartition to merge many small files; adjust number based on your cluster and data size.
logs_df = logs_df.repartition(10)

### Step 2: Extract Relevant Information

Player Information

In [3]:
from pyspark.sql.functions import regexp_extract, transform, expr, regexp_replace, col, explode, split, lit
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

# Extract player names and ratings
logs_df = logs_df.withColumn("player1", logs_df["players"].getItem(0))
logs_df = logs_df.withColumn("player2", logs_df["players"].getItem(1))

# For player1_rating_before and player2_rating_before
logs_df = logs_df.withColumn("player1_rating_before", regexp_extract("log", r"\{\}'s rating: (\d+)", 1))
logs_df = logs_df.withColumn("player2_rating_before", regexp_extract("log", r"\{\}'s rating: (\d+)", 1))

# For player1_rating_after and player2_rating_after
logs_df = logs_df.withColumn("player1_rating_after", regexp_extract("log", r"\{\}'s rating: \d+ → (\d+)", 1))
logs_df = logs_df.withColumn("player2_rating_after", regexp_extract("log", r"\{\}'s rating: \d+ → (\d+)", 1))

Pokémon, Moves, and Items

In [4]:
# Extract all Pokémon for each player
logs_df = logs_df.withColumn("player1_pokemon_array", 
                             expr("regexp_extract_all(log, '\\\\|poke\\\\|p1\\\\|([^\\\\|]+)', 1)"))
logs_df = logs_df.withColumn("player2_pokemon_array", 
                             expr("regexp_extract_all(log, '\\\\|poke\\\\|p2\\\\|([^\\\\|]+)', 1)"))

# Then clean each array element to keep only the Pokémon name
logs_df = logs_df.withColumn("player1_pokemon_array", 
                             transform("player1_pokemon_array", lambda x: regexp_replace(x, ", L\\d+.*", "")))
logs_df = logs_df.withColumn("player2_pokemon_array", 
                             transform("player2_pokemon_array", lambda x: regexp_replace(x, ", L\\d+.*", "")))


# Extract all moves used by each player's Pokémon (will parse this later)
logs_df = logs_df.withColumn("player1_moves_raw", 
                             expr("regexp_extract_all(log, '\\\\|move\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
logs_df = logs_df.withColumn("player2_moves_raw", 
                             expr("regexp_extract_all(log, '\\\\|move\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))


# Extract items used by each Pokémon
# logs_df = logs_df.withColumn("player1_item", regexp_extract("log", r"\|enditem\|p1b: [^\|]+", 1))
# logs_df = logs_df.withColumn("player2_item", regexp_extract("log", r"\|enditem\|p2b: [^\|]+", 1))

# # Extract Tera type used by each Pokémon
# logs_df = logs_df.withColumn("player1_tera", regexp_extract("log", r"\|-terastallize\|p1a: [^\|]+\|([^\|]+)", 1))
# logs_df = logs_df.withColumn("player2_tera", regexp_extract("log", r"\|-terastallize\|p2a: [^\|]+\|([^\|]+)", 1))

# Extract items used by each Pokémon - improved version
logs_df = logs_df.withColumn("player1_items_raw", 
                           expr("regexp_extract_all(log, '\\\\|item\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
logs_df = logs_df.withColumn("player2_items_raw", 
                           expr("regexp_extract_all(log, '\\\\|item\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))

# Also capture enditem events which show when items are consumed/lost
logs_df = logs_df.withColumn("player1_enditems_raw", 
                           expr("regexp_extract_all(log, '\\\\|enditem\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
logs_df = logs_df.withColumn("player2_enditems_raw", 
                           expr("regexp_extract_all(log, '\\\\|enditem\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))

# Extract Tera type used by each Pokémon - improved version
logs_df = logs_df.withColumn("player1_tera_raw", 
                           expr("regexp_extract_all(log, '\\\\|-terastallize\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
logs_df = logs_df.withColumn("player2_tera_raw", 
                           expr("regexp_extract_all(log, '\\\\|-terastallize\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))



Weather, Forfeits, Status Effects

In [5]:
# Extract weather changes
logs_df = logs_df.withColumn("weather", regexp_extract("log", r"\|weather\|([^\|]+)", 1))

# Extract forfeit information
logs_df = logs_df.withColumn("forfeit", regexp_extract("log", r"\|message\|([^\|]+ forfeited)", 1))

# Extract status effects like confusion, sleep, etc.
logs_df = logs_df.withColumn("status_effects", regexp_extract("log", r"\|start\|p1a: [^\|]+\|([^\|]+)", 1))


Match Outcome

In [6]:
# Extract the winner
logs_df = logs_df.withColumn("winner", regexp_extract("log", r"\|win\|([^\|]+)", 1))


Lets check our work!

In [7]:
logs_df.show(5)

+--------------------+---------------+--------------------+--------------------+--------+--------------------+-------+------+----------+-----+----------------+--------------+---------------------+---------------------+--------------------+--------------------+---------------------+---------------------+--------------------+--------------------+-----------------+-----------------+--------------------+--------------------+--------------------+--------------------+-------+-------+--------------+----------------+
|              format|       formatid|                  id|                 log|password|             players|private|rating|uploadtime|views|         player1|       player2|player1_rating_before|player2_rating_before|player1_rating_after|player2_rating_after|player1_pokemon_array|player2_pokemon_array|   player1_moves_raw|   player2_moves_raw|player1_items_raw|player2_items_raw|player1_enditems_raw|player2_enditems_raw|    player1_tera_raw|    player2_tera_raw|weather|forfeit|sta

That's a bit hard to read, lets convert those top 5 rows to pandas and print it out.

In [8]:
import pandas as pd

# Convert to Pandas DataFrame (only do this for small result sets!)
pandas_df = logs_df.limit(5).toPandas()

# Set pandas display options to show full arrays and columns
# original_max_colwidth = pd.get_option('display.max_colwidth')

# Temporarily set max_colwidth to None for displaying
# pd.set_option('display.max_colwidth', None)

# Display only the pokemon array columns
print("Player 1 Pokemon Arrays:")
print(pandas_df[['player1_pokemon_array']])
print("\nPlayer 2 Pokemon Arrays:")
print(pandas_df[['player2_pokemon_array']])

# Reset to original setting
# pd.set_option('display.max_colwidth', original_max_colwidth)


# Display the pandas DataFrame
pandas_df

Player 1 Pokemon Arrays:
                               player1_pokemon_array
0  [Calyrex-Shadow, Ogerpon-Hearthflame, Indeedee...
1  [Ninetales-Alola, Kyurem-White, Flutter Mane, ...
2  [Eternatus, Rillaboom, Amoonguss, Flutter Mane...
3  [Calyrex-Shadow, Grimmsnarl, Incineroar, Clefa...
4  [Amoonguss, Scream Tail, Gothitelle, Koraidon,...

Player 2 Pokemon Arrays:
                               player2_pokemon_array
0  [Eternatus, Grimmsnarl, Urshifu-*, Incineroar,...
1  [Eternatus, Rillaboom, Clefairy, Ditto, Toxape...
2  [Glastrier, Grimmsnarl, Rhydon, Amoonguss, Ete...
3  [Miraidon, Grimmsnarl, Farigiraf, Skeledirge, ...
4  [Glimmora, Maushold, Incineroar, Lugia, Grimms...


,format,formatid,id,log,password,players,private,rating,uploadtime,views,...,player1_items_raw,player2_items_raw,player1_enditems_raw,player2_enditems_raw,player1_tera_raw,player2_tera_raw,weather,forfeit,status_effects,winner
0,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2312503315,|j|☆Toger220\n|j|☆Banthejuggalo\n|t:|174111821...,None,"[Toger220, Banthejuggalo]",0,1413.0,1741119044,5,...,[],[],[],[],[],[|-terastallize|p2b: Eternatus|Water\n],,,,Banthejuggalo\n
1,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2312569807,|j|☆Mitch3204\n|j|☆Firekeeper33\n|t:|174112461...,None,"[Mitch3204, Firekeeper33]",0,1202.0,1741125243,16,...,[],[],[],[],[|-terastallize|p1a: Kyurem|Ice\n],[|-terastallize|p2a: Eternatus|Dark\n],,,,Firekeeper33\n
2,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2310685012,|j|☆yoshikawawa\n|j|☆Chibiterasuu\n|t:|1740884...,None,"[yoshikawawa, Chibiterasuu]",0,NaN,1740886914,15,...,[],[],[],[],[|-terastallize|p1a: cute dog|Dark\n],[],,,,yoshikawawa\n
3,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2314582184,|j|☆npachon1231\n|j|☆VTexan007\n|t:|1741369614...,None,"[npachon1231, VTexan007]",0,NaN,1741370196,2,...,[],[],[],[],[|-terastallize|p1a: Calyrex|Fairy\n],[|-terastallize|p2a: Wo-Chien|Poison\n],,,,VTexan007\n
4,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2308862334,|j|☆schneewittchenxj\n|j|☆Qwertytraining\n|t:|...,None,"[schneewittchenxj, Qwertytraining]",0,1121.0,1740655386,7,...,[],[],[],[],[|-terastallize|p1a: Sleepy Joe|Dark\n],[|-terastallize|p2a: Maushold|Normal\n],,,,Qwertytraining\n


In [9]:
# # Extract all moves used by each player's Pokémon
# logs_df = logs_df.withColumn("player1_moves_raw", 
#                              expr("regexp_extract_all(log, '\\\\|move\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
# logs_df = logs_df.withColumn("player2_moves_raw", 
#                              expr("regexp_extract_all(log, '\\\\|move\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))

# Show the raw extracted move patterns to verify
# print("Raw move patterns for player 1:")
# pd.set_option('display.max_colwidth', None)
# print(logs_df.select("player1_moves_raw").limit(2).toPandas())

### Extract Pokemon Moves

Previously we built a column with all the data for a pokemon and its moves.

Now we will create a new df for the moves.

1. Explode the moves ie create a new row for each move
2. Extract the pokemon for each move, and its moves
3. Group each pokemon by the battle it took place in (because we seperated the dfs into a player 1 and player 2 we don't need to worry about 2 players using the same pokemon accidentally being joined together)
4. Convert to a map

In [10]:
from pyspark.sql.functions import regexp_extract, transform, expr, regexp_replace, col, explode, split, lit
from pyspark.sql.functions import collect_list, struct, map_from_entries, collect_set


# Explode the arrays to work with individual move patterns
p1_moves_df = logs_df.select("id", explode("player1_moves_raw").alias("move_pattern"))
p2_moves_df = logs_df.select("id", explode("player2_moves_raw").alias("move_pattern"))

# Extract Pokémon and move from the patterns using regexp_extract
p1_moves_df = p1_moves_df.withColumn("pokemon", 
                                     regexp_extract("move_pattern", "\\|move\\|p1[ab]: ([^\\|]+)\\|", 1))
p1_moves_df = p1_moves_df.withColumn("move", 
                                     regexp_extract("move_pattern", "\\|move\\|p1[ab]: [^\\|]+\\|([^\\|]+)", 1))

p2_moves_df = p2_moves_df.withColumn("pokemon", 
                                     regexp_extract("move_pattern", "\\|move\\|p2[ab]: ([^\\|]+)\\|", 1))
p2_moves_df = p2_moves_df.withColumn("move", 
                                     regexp_extract("move_pattern", "\\|move\\|p2[ab]: [^\\|]+\\|([^\\|]+)", 1))

# Group by battle ID and Pokémon to get all moves used by each Pokémon. 
# We use set to remove duplicates (ie pokemon using same move more than once per match)
p1_moves_by_pokemon = p1_moves_df.groupBy("id", "pokemon").agg(collect_set("move").alias("moves"))
p2_moves_by_pokemon = p2_moves_df.groupBy("id", "pokemon").agg(collect_set("move").alias("moves"))

# Convert to a map structure for easier joining
p1_moves_map = p1_moves_by_pokemon.groupBy("id").agg(
    map_from_entries(collect_list(struct("pokemon", "moves"))).alias("player1_pokemon_moves")
)
p2_moves_map = p2_moves_by_pokemon.groupBy("id").agg(
    map_from_entries(collect_list(struct("pokemon", "moves"))).alias("player2_pokemon_moves")
)

# Join back to the main dataframe
logs_df = logs_df.join(p1_moves_map, "id", "left")
logs_df = logs_df.join(p2_moves_map, "id", "left")

# Display the results to verify
pd.set_option('display.max_colwidth', None)
sample_df = logs_df.select("id", "player1_pokemon_moves", "player2_pokemon_moves").limit(2)
print(sample_df.toPandas())

                           id  \
0  gen9vgc2025regg-2312916112   
1  gen9vgc2025regg-2313234231   

                                                                                                                                                                                                                         player1_pokemon_moves  \
0                                                                        {'岩椪': ['Spiky Shield', 'Ivy Cudgel'], '怪物': ['Parabolic Charge', 'Calm Mind', 'Protect', 'Dragon Pulse'], '宇宙第一': ['Will-O-Wisp', 'Knock Off'], '智能猩': ['Bulldoze']}   
1  {'Fletchinder': ['Dual Wingbeat', 'Will-O-Wisp', 'Feather Dance', 'Tailwind'], 'Fezandipiti': ['Poison Gas', 'Roost'], 'Bulu At Home': ['Pollen Puff', 'Strength Sap', 'Protect'], 'Eternatus': ['Fire Spin', 'Venoshock', 'Cosmic Power']}   

                                                                                                                                                                             

#### Cache the logs df so we don't have to do these expensive computes

In [11]:
# Cache the DataFrame after the expensive move mapping operations
logs_df = logs_df.cache()

### Process Items

Each pokemon holds an item. This item is not revealed unless some game event leads to it being revealed.

To solve this, we parse all item logs to match items to a pokemon. This could be an item knocked off mid turn, or an end turn item useage like berry consumption.

In [13]:

from pyspark.sql.functions import regexp_extract, transform, expr, regexp_replace, col, explode, split, lit
from pyspark.sql.functions import collect_list, struct, map_from_entries, collect_set, explode_outer

# Process both enditem events and item events
# First, extract all item-related patterns from the logs
logs_df = logs_df.withColumn("player1_item_patterns", 
                           expr("regexp_extract_all(log, '\\\\|-item\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
logs_df = logs_df.withColumn("player2_item_patterns", 
                           expr("regexp_extract_all(log, '\\\\|-item\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))

# Also capture enditem events which show when items are consumed/lost
logs_df = logs_df.withColumn("player1_enditem_patterns", 
                           expr("regexp_extract_all(log, '\\\\|-enditem\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
logs_df = logs_df.withColumn("player2_enditem_patterns", 
                           expr("regexp_extract_all(log, '\\\\|-enditem\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))

# Process item events for player 1
p1_item_df = logs_df.select("id", explode_outer("player1_item_patterns").alias("item_pattern"))
p1_item_df = p1_item_df.withColumn("pokemon", 
                                 regexp_extract("item_pattern", "\\|-item\\|p1[ab]: ([^\\|]+)\\|", 1))
p1_item_df = p1_item_df.withColumn("item", 
                                 regexp_extract("item_pattern", "\\|-item\\|p1[ab]: [^\\|]+\\|([^\\|]+)", 1))

# Process enditem events for player 1
p1_enditem_df = logs_df.select("id", explode_outer("player1_enditem_patterns").alias("enditem_pattern"))
p1_enditem_df = p1_enditem_df.withColumn("pokemon", 
                                       regexp_extract("enditem_pattern", "\\|-enditem\\|p1[ab]: ([^\\|]+)\\|", 1))
p1_enditem_df = p1_enditem_df.withColumn("item", 
                                       regexp_extract("enditem_pattern", "\\|-enditem\\|p1[ab]: [^\\|]+\\|([^\\|]+)", 1))

# Process item events for player 2
p2_item_df = logs_df.select("id", explode_outer("player2_item_patterns").alias("item_pattern"))
p2_item_df = p2_item_df.withColumn("pokemon", 
                                 regexp_extract("item_pattern", "\\|-item\\|p2[ab]: ([^\\|]+)\\|", 1))
p2_item_df = p2_item_df.withColumn("item", 
                                 regexp_extract("item_pattern", "\\|-item\\|p2[ab]: [^\\|]+\\|([^\\|]+)", 1))

# Process enditem events for player 2
p2_enditem_df = logs_df.select("id", explode_outer("player2_enditem_patterns").alias("enditem_pattern"))
p2_enditem_df = p2_enditem_df.withColumn("pokemon", 
                                       regexp_extract("enditem_pattern", "\\|-enditem\\|p2[ab]: ([^\\|]+)\\|", 1))
p2_enditem_df = p2_enditem_df.withColumn("item", 
                                       regexp_extract("enditem_pattern", "\\|-enditem\\|p2[ab]: [^\\|]+\\|([^\\|]+)", 1))

# Union the item and enditem dataframes for each player
p1_all_items_df = p1_item_df.union(p1_enditem_df)
p2_all_items_df = p2_item_df.union(p2_enditem_df)

# Group by battle ID and Pokémon to get items used by each Pokémon
p1_items_by_pokemon = p1_all_items_df.filter(col("pokemon").isNotNull() & col("item").isNotNull()) \
                                    .groupBy("id", "pokemon").agg(collect_set("item").alias("items"))
p2_items_by_pokemon = p2_all_items_df.filter(col("pokemon").isNotNull() & col("item").isNotNull()) \
                                    .groupBy("id", "pokemon").agg(collect_set("item").alias("items"))

# Convert to a map structure for easier joining
p1_items_map = p1_items_by_pokemon.groupBy("id").agg(
    map_from_entries(collect_list(struct("pokemon", "items"))).alias("player1_pokemon_items")
)
p2_items_map = p2_items_by_pokemon.groupBy("id").agg(
    map_from_entries(collect_list(struct("pokemon", "items"))).alias("player2_pokemon_items")
)

# Join back to the main dataframe
logs_df = logs_df.join(p1_items_map, "id", "left")
logs_df = logs_df.join(p2_items_map, "id", "left")

# Display the results to verify
sample_df = logs_df.select("id", "player1_pokemon_items", "player2_pokemon_items").limit(2)
print(sample_df.toPandas())
# // ... existing code ...

                           id           player1_pokemon_items  \
0  gen9vgc2025regg-2308834240    {'Urshifu': ['Focus Sash
']}   
1  gen9vgc2025regg-2308833635  {'Chien-Pao': ['Focus Sash
']}   

                   player2_pokemon_items  
0  {'Flutter Mane': ['Booster Energy
']}  
1                                   None  


Lets do a pandas print for readbility

In [14]:
# Convert to Pandas DataFrame (only do this for small result sets!)
pandas_df = logs_df.limit(5).toPandas()


# Display the pandas DataFrame
pandas_df

id                  format         formatid  \
0  gen9vgc2025regg-2308834240  [Gen 9] VGC 2025 Reg G  gen9vgc2025regg   
1  gen9vgc2025regg-2308835437  [Gen 9] VGC 2025 Reg G  gen9vgc2025regg   
2  gen9vgc2025regg-2308837983  [Gen 9] VGC 2025 Reg G  gen9vgc2025regg   
3  gen9vgc2025regg-2308833635  [Gen 9] VGC 2025 Reg G  gen9vgc2025regg   
4  gen9vgc2025regg-2308836794  [Gen 9] VGC 2025 Reg G  gen9vgc2025regg   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

### Extracting Tera Information

In [15]:
from pyspark.sql.functions import regexp_extract, transform, expr, regexp_replace, col, explode, split, lit
from pyspark.sql.functions import collect_list, struct, map_from_entries, collect_set, explode_outer

# Process terastallize events for both players
# First, extract all terastallize patterns from the logs
logs_df = logs_df.withColumn("player1_tera_patterns", 
                           expr("regexp_extract_all(log, '\\\\|-terastallize\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
logs_df = logs_df.withColumn("player2_tera_patterns", 
                           expr("regexp_extract_all(log, '\\\\|-terastallize\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))

# Process terastallize events for player 1
p1_tera_df = logs_df.select("id", explode_outer("player1_tera_patterns").alias("tera_pattern"))
p1_tera_df = p1_tera_df.withColumn("pokemon", 
                                 regexp_extract("tera_pattern", "\\|-terastallize\\|p1[ab]: ([^\\|]+)\\|", 1))
p1_tera_df = p1_tera_df.withColumn("tera_type", 
                                 regexp_extract("tera_pattern", "\\|-terastallize\\|p1[ab]: [^\\|]+\\|([^\\|\\n]+)", 1))

# Process terastallize events for player 2
p2_tera_df = logs_df.select("id", explode_outer("player2_tera_patterns").alias("tera_pattern"))
p2_tera_df = p2_tera_df.withColumn("pokemon", 
                                 regexp_extract("tera_pattern", "\\|-terastallize\\|p2[ab]: ([^\\|]+)\\|", 1))
p2_tera_df = p2_tera_df.withColumn("tera_type", 
                                 regexp_extract("tera_pattern", "\\|-terastallize\\|p2[ab]: [^\\|]+\\|([^\\|\\n]+)", 1))

# Filter out null values and create maps
p1_tera_map = p1_tera_df.filter(col("pokemon").isNotNull() & col("tera_type").isNotNull()) \
                       .groupBy("id") \
                       .agg(map_from_entries(collect_list(struct("pokemon", "tera_type"))).alias("player1_pokemon_tera"))

p2_tera_map = p2_tera_df.filter(col("pokemon").isNotNull() & col("tera_type").isNotNull()) \
                       .groupBy("id") \
                       .agg(map_from_entries(collect_list(struct("pokemon", "tera_type"))).alias("player2_pokemon_tera"))

# Join back to the main dataframe
logs_df = logs_df.join(p1_tera_map, "id", "left")
logs_df = logs_df.join(p2_tera_map, "id", "left")

# Display the results to verify
sample_df = logs_df.select("id", "player1_pokemon_tera", "player2_pokemon_tera").limit(5)
print(sample_df.toPandas())

                           id       player1_pokemon_tera  \
0  gen9vgc2025regg-2308833635        {'Kyogre': 'Water'}   
1  gen9vgc2025regg-2308834240        {'Entei': 'Normal'}   
2  gen9vgc2025regg-2308835437       {'Calyrex': 'Water'}   
3  gen9vgc2025regg-2308836794       {'Koraidon': 'Fire'}   
4  gen9vgc2025regg-2308837983  {'Flutter Mane': 'Fairy'}   

      player2_pokemon_tera  
0                     None  
1                     None  
2                     None  
3  {'Raging Bolt': 'Dark'}  
4    {'Reshiram': 'Grass'}  


In [16]:
# Convert to Pandas DataFrame (only do this for small result sets!)
pandas_df = logs_df.limit(5).toPandas()


# Display the pandas DataFrame
pandas_df

id                  format         formatid  \
0  gen9vgc2025regg-2308834240  [Gen 9] VGC 2025 Reg G  gen9vgc2025regg   
1  gen9vgc2025regg-2308835437  [Gen 9] VGC 2025 Reg G  gen9vgc2025regg   
2  gen9vgc2025regg-2308837983  [Gen 9] VGC 2025 Reg G  gen9vgc2025regg   
3  gen9vgc2025regg-2308833635  [Gen 9] VGC 2025 Reg G  gen9vgc2025regg   
4  gen9vgc2025regg-2308836794  [Gen 9] VGC 2025 Reg G  gen9vgc2025regg   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

### Save to parquet

We will end this exploration and save our results to parquet so we can reuse (and also put other functions so we can copy and paste them for later)

In [17]:
# ... existing code ...

# Save the processed data to parquet
from datetime import datetime
import os

# Create a timestamp for the processed data
process_date = datetime.now().strftime("%Y-%m-%d")

# Define the output paths
processed_data_path = f"../../data/processed/replays/{process_date}"
processed_ids_path = f"../../data/processed/replay_ids/{process_date}"

# Ensure directories exist
os.makedirs(os.path.dirname(processed_data_path), exist_ok=True)
os.makedirs(os.path.dirname(processed_ids_path), exist_ok=True)

# Save the full processed dataframe as parquet
logs_df.write.mode("overwrite").parquet(processed_data_path)
print(f"Saved processed data to {processed_data_path}")

# Save just the IDs of processed replays for future reference
id_df = logs_df.select("id")
id_df.write.mode("overwrite").parquet(processed_ids_path)
print(f"Saved processed IDs to {processed_ids_path}")

# Example of how to use these IDs for incremental processing in the future:
print("\nExample code for future incremental processing:")
print("""
# Load previously processed IDs
processed_ids = spark.read.parquet("../../data/processed/replay_ids/*")

# Load new raw data
new_logs_df = spark.read.option("recursiveFileLookup", "true").option("pathGlobFilter", "*.json").json("../../data/replays/gen9vgc2025regg/")

# Filter out already processed replays
new_logs_df = new_logs_df.join(processed_ids, "id", "left_anti")

# Process only the new data
# ... processing code ...

# Append to existing processed data
new_logs_df.write.mode("append").parquet("../../data/processed/replays/YYYY-MM-DD")

# Update the processed IDs
new_logs_df.select("id").write.mode("append").parquet("../../data/processed/replay_ids/YYYY-MM-DD")
""")

# For testing, let's see how many replays we processed
replay_count = logs_df.count()
print(f"\nProcessed {replay_count} replays in this run")

Saved processed data to ../../data/processed/replays/2025-03-08
Saved processed IDs to ../../data/processed/replay_ids/2025-03-08

Example code for future incremental processing:

# Load previously processed IDs
processed_ids = spark.read.parquet("../../data/processed/replay_ids/*")

# Load new raw data
new_logs_df = spark.read.option("recursiveFileLookup", "true").option("pathGlobFilter", "*.json").json("../../data/replays/gen9vgc2025regg/")

# Filter out already processed replays
new_logs_df = new_logs_df.join(processed_ids, "id", "left_anti")

# Process only the new data
# ... processing code ...

# Append to existing processed data
new_logs_df.write.mode("append").parquet("../../data/processed/replays/YYYY-MM-DD")

# Update the processed IDs
new_logs_df.select("id").write.mode("append").parquet("../../data/processed/replay_ids/YYYY-MM-DD")


Processed 24989 replays in this run
